In [19]:
import pandas as pd
import numpy as np
from read_path_module import read_data_relative_path
from write_path_module import write_data_relative_path

## Importing 2003 - 2006 Data

In [20]:
# Read in txt file
df = read_data_relative_path('./data/CDC_Natality/2003_2006.txt', data_type='txt_tab')

# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2003_2006.csv', data_type='csv')
del df

# Import in data and rename
df_2003_2006 = read_data_relative_path('./data/CDC_Natality/2003_2006.csv', data_type='csv')
df_2003_2006.sample(3)

,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age
25248,NaN,2006.0,2006.0,May,5.0,Minnesota,27.0,"Hennepin County, MN",27053.0,1408.0,29.01,38.68
11580,NaN,2004.0,2004.0,July,7.0,New Jersey,34.0,"Union County, NJ",34039.0,615.0,29.61,38.72
23021,NaN,2006.0,2006.0,January,1.0,Texas,48.0,"Randall County, TX",48381.0,111.0,25.94,38.46


## Importing 2007 - 2018 Data

In [21]:
# Create path list for all txt files
lst_paths = ['./data/CDC_Natality/'+str(i)+'.txt' for i in range(2007,2018+1)]

# Read in first csv
df = read_data_relative_path('./data/CDC_Natality/2007.txt', data_type='txt_tab').head(1)

# Delete data to create empty, but properly columned dataframe
df = df[0:0]

# Iterate through path names and append them to empty dataframe above
for path in lst_paths:
    temp = read_data_relative_path(path, data_type='txt_tab')
    df = pd.concat([df, temp])
    del temp

# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2007_2018.csv', data_type='csv')
del df

# Import in data and rename
df_2007_2018 = read_data_relative_path('./data/CDC_Natality/2007_2018.csv', data_type='csv')
df_2007_2018.sample(5)

,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Average OE Gestational Age
89319,NaN,2017.0,2017.0,November,11.0,Minnesota,27.0,"Hennepin County, MN",27053.0,1276,31.07,38.86,38.73
32575,NaN,2010.0,2010.0,December,12.0,North Carolina,37.0,"Harnett County, NC",37085.0,Missing County,Missing County,Missing County,Missing County
36176,NaN,2011.0,2011.0,May,5.0,Texas,48.0,"Bell County, TX",48027.0,491,25.80,38.45,38.28
42166,NaN,2012.0,2012.0,February,2.0,New Mexico,35.0,"Bernalillo County, NM",35001.0,667,27.24,38.85,38.51
10191,NaN,2008.0,2008.0,March,3.0,Texas,48.0,"Unidentified Counties, TX",48999.0,5663,25.33,38.34,38.27


## Combine the two datasets

In [22]:
# Check to see if column names match
col_2003 = df_2003_2006.columns.to_list()
col_2007 = df_2007_2018.columns.to_list()

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

print('The difference between the two datasets is the column: ', Diff(col_2007, col_2003)) 
# print('The difference between the two datasets is the column: ', Diff(col_2003, col_2007)) # No difference

# Drop this column from the 2007 DF
df_2007_2018.drop(columns=['Average OE Gestational Age'], inplace=True)

# Concatenate the two columns
df = pd.concat([df_2003_2006, df_2007_2018])
df.sample(5)

The difference between the two datasets is the column:  ['Average OE Gestational Age']


,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age
3615,NaN,2007.0,2007.0,June,6.0,Iowa,19.0,"Johnson County, IA",19103.0,130,29.12,39.08
79420,NaN,2016.0,2016.0,August,8.0,Wisconsin,55.0,"Waukesha County, WI",55133.0,363,30.92,38.74
15924,NaN,2008.0,2008.0,December,12.0,Kansas,20.0,"Sedgwick County, KS",20173.0,728,26.55,38.48
59874,NaN,2014.0,2014.0,April,4.0,Michigan,26.0,"Berrien County, MI",26021.0,161,26.60,39.22
85394,NaN,2017.0,2017.0,May,5.0,Ohio,39.0,"Mahoning County, OH",39099.0,207,27.53,38.45


In [23]:
#There are some rows that:
    # are footnotes about the dataset (i.e. everything is null besides the note column)
    # are totals of each year, month, and state grouping
# Here we will remove those rows
# Before that we will check to make sure we are just removing those ones
    
# number of rows of full dataset
num_total = df.shape[0]

# number of rows where everything is null besides note column (the pesky footnote rows)
num_all_null = df[df.iloc[:, -11:].isnull().all(axis=1)].shape[0]

# number of rows where note is null (this is a valid entry)
num_note_null = df[df['Notes'].isna()].shape[0]

# number of rows where note is 'Total' (this is a valid entry)
num_note_total = df[df['Notes']=='Total'].shape[0]

# sum up these row counts and see if they match
print(num_total, num_all_null + num_note_null + num_note_total, num_all_null, num_note_null, num_note_total)

# since they do we will concatenate everything except for the pesky footnote rows
df = df[df['Notes'].isna()]

128638 128638 737 117888 10013


## - Change the 'Missing County' label to NA

In [24]:
col_lst = df.columns.to_list()

for col in col_lst:
    df.loc[df[col] == 'Missing County', col] = np.nan
del col_lst

/Users/michaellink/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


## - Convert appropriate columns to int

In [25]:
# df['Month Code'] = df['Month Code'].astype(int)
col_lst = ['Year', 'Year Code', 'Month Code', 'State Code', 'County Code']

for col in col_lst:
    df[col] = df[col].astype(int)

### - Rename columns

In [26]:
df.rename(columns = {
    "Year": "Birth Year",
    "Month": "Birth Month",
    "Month Code": "Birth Month Code"},
         inplace = True)

### - Remove NA rows where appropriate

In [27]:
print('There were ',df['Births'].isna().sum(), ' Births NA rows.')

# Remove rows where birth data is not present
df = df[-df['Births'].isna()]

print( 'There are now ', df['Births'].isna().sum(), ' Births NA rows.')

There were  4824  Births NA rows.
There are now  0  Births NA rows.


### - Create pandas datetime column

In [28]:
df['Birth Date'] = pd.to_datetime(df['Birth Year'].astype(str) + '-' + df['Birth Month Code'].astype(str) + '-' + '15', format='%Y-%m-%d')
df.head(2)

,Notes,Birth Year,Year Code,Birth Month,Birth Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Birth Date
0,NaN,2003,2003,January,1,Alabama,1,"Baldwin County, AL",1003,131,26.77,37.97,2003-01-15
1,NaN,2003,2003,January,1,Alabama,1,"Calhoun County, AL",1015,124,25.32,38.63,2003-01-15


### - Backtrack to calculate conception date
- At a high level
    - A menstrual cycle starts
    - Two weeks later ovulation occurs
    - Sparks fly and conception occurs
    - ~38 weeks later parents sleep cycles are forever changed...
    
    
- Within the dataset LMP gestational age is given. This is the estimate of the last menstrual cycle


- To calculate conception date we:
    - Take date of birth (+/- 15 days) then subtract LMP Age then add 2 weeks to ovulation

In [29]:
# Transforming the gestational age into a timedelta for time arithmetic
lst_time_intrval = [pd.Timedelta(float(x)*7, unit='days') for x in df['Average LMP Gestational Age']]
    
# Date of birth - LMP Age + 2 weeks to ovulation
df['Conception Date'] = df['Birth Date'] - pd.Series(lst_time_intrval).values + pd.Timedelta(2, unit='W')

# Standard 38 weeks
df['Conception Date Standard'] = df['Birth Date'] - pd.Timedelta(38, unit='W')

df[['Birth Date','Conception Date', 'Conception Date Standard']].head(2)

,Birth Date,Conception Date,Conception Date Standard
0,2003-01-15,2002-05-08 05:02:24,2002-04-24
1,2003-01-15,2002-05-03 14:09:36,2002-04-24


### - Create Conception Year, Month, and Month Code Columns

In [30]:
df['Conception Year'] = pd.DatetimeIndex(df['Conception Date']).year

df['Conception Month'] = pd.DatetimeIndex(df['Conception Date']).month_name()

df['Conception Month Code'] = pd.DatetimeIndex(df['Conception Date']).month

df['Conception Year Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).year

df['Conception Month Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).month_name()

df['Conception Month Code Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).month


# Convert to integers
col_lst = ['Conception Year', 'Conception Month Code', 'Conception Year Standard', 'Conception Month Code Standard']

for col in col_lst:
    df[col] = df[col].astype(int)

### - Clean, reorder, and create new columns

In [31]:
df['State Acronym'] = df['County'].str[-2:]
df[['County','temp']] = df['County'].str.split(',', expand=True)
df.drop(columns=['temp'], inplace=True)

df.sample(3)

df.columns.to_list()

col_lst = ['Birth Year', 'Conception Year', 'Conception Year Standard', 'Birth Month', 'Conception Month',
           'Conception Month Standard', 'Birth Month Code', 'Conception Month Code', 'Conception Month Code Standard',
           'Birth Date', 'Conception Date', 'Conception Date Standard', 'State', 'State Acronym',
           'State Code','County','County Code', 'Births','Average Age of Mother',
           'Average LMP Gestational Age','Notes']

df = df[col_lst]
del col_lst
print(df.shape)
df.sample(3)

(113064, 21)


,Birth Year,Conception Year,Conception Year Standard,Birth Month,Conception Month,Conception Month Standard,Birth Month Code,Conception Month Code,Conception Month Code Standard,Birth Date,...,Conception Date Standard,State,State Acronym,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Notes
58686,2014,2013,2013,February,June,May,2,6,5,2014-02-15,...,2013-05-25,Ohio,OH,39,Hamilton County,39061,859,28.16,38.49,NaN
32555,2010,2010,2010,December,April,March,12,4,3,2010-12-15,...,2010-03-24,New York,NY,36,Saratoga County,36091,179,29.41,38.76,NaN
62164,2014,2013,2013,July,October,October,7,10,10,2014-07-15,...,2013-10-22,South Carolina,SC,45,Berkeley County,45015,252,27.91,38.58,NaN


## - Write result to csv

In [32]:
# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2003_2018.csv', data_type='csv')
# del df